# Interactive Brokers Movement Upload

In [58]:
import os
import datetime
import time
from pathlib import Path
import yaml
import sys
import importlib
import pandas as pd

sys.path.append("..")

 
Path("a/b/coub.csv").stem

'coub'

In [ ]:
from salduba.ib_tws_proxy.backing_db.model import ContractRecord

ContractRecord.fields()

: 

In [54]:
import dataclasses as dc
from dataclasses import dataclass
from typing import ClassVar, Self


@dataclass
class Base:
  table: ClassVar[str]
  counter: ClassVar[str]
  selector: ClassVar[str]
  inserter: ClassVar[str]
  value_inserter: ClassVar[str]
  updater: ClassVar[str]
  byIdDeleter: ClassVar[str]  
  s1: str
  s2: int

  @classmethod
  def fields(cls: type[Self]) -> list[str]:
      return [f.name for f in dc.fields(cls)]
    
  @classmethod
  def __init_subclass__(cls: type[Self], **kwargs) -> None:  # type: ignore
    if Base not in cls.__bases__:
      raise Exception(f"Record must be directly subclassed from: {cls.__name__}")
    for required in ['table']:
      if not getattr(cls, required):
        raise TypeError(f"Can't instantiate abstract class {cls.__name__} without {required} attribute defined")
      cls.counter = "Select count(id) from " + cls.table
      cls.selector = "select " + ", ".join(cls.fields()) + " from " + cls.table
      cls.inserter = "Insert into " + cls.table + " (" + ", ".join(cls.fields()) + ") "
      cls.value_inserter = cls.inserter + "values (" + ", ".join(["?"] * len(cls.fields())) + ");"
      cls.updater = "Update " + cls.table + " set " + ", ".join([f"{k} = ?" for k in cls.fields()])
      cls.byIdDeleter = "Delete from {} where id == ?".format(cls.table)
    return super().__init_subclass__(**kwargs)
  
  
  @classmethod
  def l_fields(cls) -> list[str]:
    print(f"Fields for {cls.__name__}")
    # [filename for path in dirs for filename in os.listdir(path)]
    baseFields: list[str] = [f for c in cls.__bases__ for f in c.fields()] if cls.__name__ != 'Base' else []
    ownFields: list[str] = [k for k in cls.__annotations__]
    return baseFields + (ownFields  if cls.__name__ != 'Base' else ownFields[1:])


@dataclass
class TST(Base):
  table: ClassVar[str] = "TST_T"
  aStr: str
  bInt: int

@dataclass
class BB(TST):
  downA: str
  downB: str

bb = BB("s1", 1, "aStr", 2, "downA", "downB")

[bb.table, bb.selector, bb.__dict__]

BB.__bases__


Exception: Record must be directly subclassed from: BB

In [19]:
typeTable = {
  'Equity': 'STK'
}


def split_ticker(ticker: str) -> list[str]:
  cols = ticker.split(' ')
  cols.append(typeTable.get(cols[-1], cols[-1]))
  return cols
  
def _fill_in(frame: pd.DataFrame) -> pd.DataFrame:
    frame['Symbol'], frame['Country'], frame['RawType'], frame['IbkType'] = zip(*frame.index.map(split_ticker))
    return frame.sort_values('Ticker')
  
def read_excel(movements_path: str, sheet: str = 'Movements') -> pd.DataFrame:
    movementsPD: pd.DataFrame = pd.read_excel(
      movements_path,
      sheet,
#      usecols=['Ticker', 'ID ISIN', 'Nombre', 'Trade', 'Currency', 'Exchange', 'Exchange 2'],
      index_col='Ticker',
      keep_default_na=True)
    return _fill_in(movementsPD)

NameError: name 'pd' is not defined

In [60]:
f = read_excel('../tests/resources/Cervino Rebalance 240503.xlsx', 'Movements')
f[0:5]

,ID ISIN,Nombre,Trade,Currency,Exchange,Exchange 2,Unnamed: 7,Co,Curny,Exch,Symbol,Country,RawType,IbkType
Ticker,,,,,,,,,,,,,,
2875 JP Equity,JP3613000003,Toyo Suisan Kaisha Ltd,600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2875,JP,Equity,STK
3774 JP Equity,JP3152820001,Internet Initiative Japan Inc,200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3774,JP,Equity,STK
4063 JP Equity,JP3371200001,Shin-Etsu Chemical Co Ltd,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4063,JP,Equity,STK
4186 JP Equity,JP3571800006,Tokyo Ohka Kogyo Co Ltd,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4186,JP,Equity,STK
4519 JP Equity,JP3519400000,Chugai Pharmaceutical Co Ltd,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4519,JP,Equity,STK


In [52]:
f.rename(columns={'Exchange 2': 'Exchange2'})
f.columns

Index(['ID ISIN', 'Nombre', 'Trade', 'Currency', 'Exchange', 'Exchange 2',
       'Symbol', 'Country', 'RawType', 'IbkType'],
      dtype='object')

In [95]:
f['Ticker'] = f.index

In [99]:
f.apply(lambda r: r, axis=1)

,ID ISIN,Nombre,Trade,Currency,Exchange,Exchange 2,Unnamed: 7,Co,Curny,Exch,Symbol,Country,RawType,IbkType,OTicker,Ticker
Ticker,,,,,,,,,,,,,,,,
2875 JP Equity,JP3613000003,Toyo Suisan Kaisha Ltd,600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2875,JP,Equity,STK,2875 JP Equity,2875 JP Equity
3774 JP Equity,JP3152820001,Internet Initiative Japan Inc,200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3774,JP,Equity,STK,3774 JP Equity,3774 JP Equity
4063 JP Equity,JP3371200001,Shin-Etsu Chemical Co Ltd,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4063,JP,Equity,STK,4063 JP Equity,4063 JP Equity
4186 JP Equity,JP3571800006,Tokyo Ohka Kogyo Co Ltd,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4186,JP,Equity,STK,4186 JP Equity,4186 JP Equity
4519 JP Equity,JP3519400000,Chugai Pharmaceutical Co Ltd,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4519,JP,Equity,STK,4519 JP Equity,4519 JP Equity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VBK GR Equity,DE000A0JL9W6,Verbio SE,-2644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VBK,GR,Equity,STK,VBK GR Equity,VBK GR Equity
VLO US Equity,US91913Y1001,Valero Energy Corp,-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VLO,US,Equity,STK,VLO US Equity,VLO US Equity
VRTX US Equity,US92532F1003,Vertex Pharmaceuticals Inc,-58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VRTX,US,Equity,STK,VRTX US Equity,VRTX US Equity


In [6]:
from decimal import Decimal

a = Decimal(33.0)

if isinstance(a, Decimal):
  print("It is a Decimal")
else:
  print("It is NOT")
    

It is a Decimal


In [8]:
movesPD = pd.read_csv('../data/cervino_rebalance_v2.csv', dtype={
    'Ticker': str,
    'Trade': int,
    'Nombre': str,
    'Symbol': str,
    'RawType': str,
    'RawCountry': str,
    'IBKType': str,
    'Currency': str,
    'Exchange': str,
    'Exchange2': str
},
                      keep_default_na=False)

movesPD[movesPD['Ticker'] == 'ASML NA Equity']

,Ticker,Trade,Nombre,Symbol,RawType,RawCountry,IBKType,Currency,Exchange,Exchange2
8,ASML NA Equity,-3,ASML Holding NV,ASML,Equity,NA,STK,EUR,AEB,


In [14]:
type(movesPD['Ticker'] == movesPD['Nombre'])

pandas.core.series.Series

In [3]:
list = os.listdir('../salduba/ib_tws_proxy/backing_db/schema/')
list.remove('meta.sql')

list
[].pop(0)

IndexError: pop from empty list

In [8]:
import re

re.search(r'\d+.sql', 'vasdf.sql') is None

True

In [2]:
import sqlite3 as sql

with open("../salduba/ib_tws_proxy/backing_db/schema/meta.sql", 'r') as script_file:
  script = script_file.read()
  
script

'create table if not exists db_info (\n  version integer primary key,\n  version_time integer,\n  at integer not NULL,\n  current BOOLEAN not NULL\n);\n'

In [4]:
db_conn = sql.connect('test.db')
crs = db_conn.cursor()

rs = crs.execute('select * from contract;').fetchall()

crs.close()
db_conn.close()

rs

[('asdf-asdf-asdf-0001',
  123456,
  234567,
  'PENDING',
  12345,
  'SYMBOL',
  'STK',
  'MARCH',
  88.99,
  'RIGHT',
  'MULTIPLIER',
  'SMART',
  'USD',
  'LOC_SYMBOL',
  'NASDAQ',
  'TRD_CLASS',
  0,
  'SEC_ID_TYPE',
  'SEC_ID',
  'Something or other',
  'Issuer',
  'The combo legs',
  'dnc_id-001')]

In [35]:
import salduba.ib_tws_proxy.backing_db.model as cmodel
importlib.reload(cmodel)

from salduba.ib_tws_proxy.backing_db.model import ContractRecord



In [36]:
ctr = ContractRecord.hydrate(*rs[0])

Found: ('asdf-asdf-asdf-0001', 123456, 234567, 'PENDING', 12345, 'SYMBOL', 'STK', 'MARCH', 88.99, 'RIGHT', 'MULTIPLIER', 'SMART', 'USD', 'LOC_SYMBOL', 'NASDAQ', 'TRD_CLASS', 0, 'SEC_ID_TYPE', 'SEC_ID', 'Something or other', 'Issuer', 'The combo legs', 'dnc_id-001')
Then the rest of parameters: {'conid': 12345, 'symbol': 'SYMBOL', 'sec_type': 'STK', 'last_trade_date_or_contract_month': 'MARCH', 'strike': 88.99, 'right': 'RIGHT', 'multiplier': 'MULTIPLIER', 'exchange': 'SMART', 'currency': 'USD', 'local_symbol': 'LOC_SYMBOL', 'primary_exchange': 'NASDAQ', 'trading_class': 'TRD_CLASS', 'include_expired': 0, 'sec_id_type': 'SEC_ID_TYPE', 'sec_id': 'SEC_ID', 'description': 'Something or other', 'issuer_id': 'Issuer', 'combo_legs_description': 'The combo legs', 'delta_neutral_contract_fk': 'dnc_id-001'}


In [46]:
datetime.datetime.now().timestamp()

1709075288.72169

In [31]:
db_conn = sql.connect('test.db')
crs = db_conn.cursor()

files = [
  "../salduba/ib_tws_proxy/backing_db/schema/meta.sql",
  "../salduba/ib_tws_proxy/backing_db/seed-data/meta.sql",
  "../salduba/ib_tws_proxy/backing_db/schema/v1.sql"
]

for f in files:
  with open(f, 'r') as file:
    script = file.read()
  crs.executescript(script)

crs.close()

In [12]:
config_path = '../configuration.yml'

with open(config_path, 'r') as config_file:
  loaded_config = yaml.safe_load(config_file)
  
loaded_config

{'logging': {'level': 'INFO'},
 'tws_server': {'host': '127.0.0.1', 'port': 7497},
 'client': {'id': 0},
 'inputs': {'movements_path': 'data/movements.csv',
  'batch': 'movements_batch'}}

In [14]:
loaded_config.update(default)
loaded_config

{'logging': {'level': 'INFO'},
 'tws_server': {'host': '127.0.0.1', 'port': '7497'},
 'client': {'id': '0'},
 'inputs': {'movements_path': 'data/movements.csv',
  'batch': 'movements::2024-02-13 13:53:31.160482'}}

## Set up Execution Environment

In [11]:
import requests as rq
import pandas as pd
import sys
import os
import importlib
import random


project_dir = os.path.dirname(os.getcwd())

sys.path.append(project_dir)


/var/folders/pc/2vy016913bs5kzpn254yq_f40000gn/T/ipykernel_37933/2111806649.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


['DU8479097', 'kemitc545', '$Secreto4PaperTrading$']

In [27]:
import ibapi as ib
from decimal import Decimal

import sqlite3
con = sqlite3.connect('../data/tickers.db')

In [10]:
from ibapi.common import ListOfContractDescription
from ibapi.common import TickerId

In [3]:
import salduba.order_model as om
importlib.reload(om)
from salduba.order_model import Order



In [7]:
from enum import StrEnum
class WKW(StrEnum):
  A = "A"
  B = "B"
  
WKW("A")
type(WKW.A)

<enum 'WKW'>

## Read data from CSV files

In [12]:
movements_path = '../data/movements.csv'
initial_path = '../data/initial.csv'
final_path = '../data/final.csv'

### Read Movements File

In [25]:
movements = pd.read_csv(movements_path, index_col='Ticker').sort_values('Ticker').drop('NOTHING', axis=1)
movements['Trade'] = movements['Trade'].map(lambda tr : int(tr.replace(',', '')))
movements['Id'] = movements['Id'].map(lambda id : int(id))

movements[movements['Nombre'] == 'flatexDEGIRO AG']
movements

,Id,Region,Nombre,Trade,Money,Currency,Min Mom 5Y10Y,Value Factor,Mom 1Y Factor,Mom+Val Adj,Correlation,GAP,100,EUR Initial Position,Initial,Final
Ticker,,,,,,,,,,,,,,,,
2222 JP Equity,97,ASIA,Kotobuki Spirits Co Ltd,300,"-600,450",JPY,15%,1.76,1.24,1.63,0.43,1.20,"2,200","2,500",NaN,NaN
3088 JP Equity,100,ASIA,MatsukiyoCocokara & Co,1900,"-5,100,550",JPY,15%,1.39,1.12,1.33,0.24,1.09,-,"1,900",NaN,NaN
3626 JP Equity,102,ASIA,TIS Inc,1500,"-5,104,500",JPY,14%,1.28,0.87,1.18,0.18,1.00,-,"1,500",NaN,NaN
3635 JP Equity,101,ASIA,Koei Tecmo Holdings Co Ltd,-200,"353,100",JPY,17%,1.52,0.69,1.31,0.26,1.06,"3,000","2,800",NaN,NaN
3774 JP Equity,106,ASIA,Internet Initiative Japan Inc,-100,"301,500",JPY,15%,1.25,1.11,1.22,0.30,0.91,"1,800","1,700",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WIRE US Equity,63,AMERICA,Encore Wire Corp,16,"-3,436",USD,17%,1.01,1.44,1.12,0.50,0.62,624,640,NaN,NaN
WLK US Equity,90,AMERICA,Westlake Corp,1026,"-137,325",USD,12%,0.95,1.18,1.01,0.67,0.33,-,"1,026",NaN,NaN
WSM US Equity,61,AMERICA,Williams-Sonoma Inc,45,"-9,121",USD,19%,1.05,1.61,1.19,0.57,0.62,633,678,NaN,NaN


In [22]:
mvm = movements.reset_index()
[(idx, row['Nombre']) for idx, row in mvm.iterrows()][0:1]

len(mvm[mvm['Trade'] == 0])

7

### Read Initial File

In [5]:
initial = pd.read_csv(initial_path, index_col='Ticker')
initial['Initial'] = initial['Initial'].map(lambda s : int(s.replace(',', '')))

initial.columns

Index(['Nombre', 'Initial'], dtype='object')

### Read Final File

In [6]:
final = pd.read_csv(final_path, index_col='Ticker')
final['Final'] = final['Final'].map(lambda s : int(s.replace(',', '')))

final.columns

Index(['Nombre', 'Final'], dtype='object')

### Check balances

In [8]:
movements['Initial'] = initial['Initial']
movements['Final'] = final['Final']

movements['Initial'] = movements['Initial'].fillna(0)
movements['Final'] = movements['Final'].fillna(0)

In [9]:
balances = movements['Initial'] + movements['Trade'] - movements['Final']

f"# Unbalanced rows {len(balances[balances != 0])}"


'# Unbalanced rows 0'

# Create Orders

In [10]:
orders = [Order(account_id, r['Id'], r['Ticker'], r['Trade']) for r in movements.reset_index()[['Id', 'Ticker', 'Trade']].to_dict('records')]

In [12]:
[r.to_dict('Test Batch', 12345, 'NASDAQ') for r in orders[0:3]]

[{'acctId': 'U13912507',
  'conid': 12345,
  'secType': '12345@NASDAQ',
  'cOID': 'Test Batch-97',
  'parentId': None,
  'orderType': 'MKT',
  'listingExchange': 'NASDAQ',
  'isSingleGroup': False,
  'outsideRTH': False,
  'side': 'BUY',
  'ticker': '2222 JP Equity',
  'tif': 'DAY',
  'referrer': 'CorvinoFundManagement2024-01-17 11:38:59.500228',
  'quantity': 300,
  'useAdaptive': False,
  'isCcyConv': False,
  'deactivated': True},
 {'acctId': 'U13912507',
  'conid': 12345,
  'secType': '12345@NASDAQ',
  'cOID': 'Test Batch-100',
  'parentId': None,
  'orderType': 'MKT',
  'listingExchange': 'NASDAQ',
  'isSingleGroup': False,
  'outsideRTH': False,
  'side': 'BUY',
  'ticker': '3088 JP Equity',
  'tif': 'DAY',
  'referrer': 'CorvinoFundManagement2024-01-17 11:38:59.500228',
  'quantity': 1900,
  'useAdaptive': False,
  'isCcyConv': False,
  'deactivated': True},
 {'acctId': 'U13912507',
  'conid': 12345,
  'secType': '12345@NASDAQ',
  'cOID': 'Test Batch-102',
  'parentId': None,
  